## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-02-14-50-00 +0000,wsj,"Afghan Quake Death Toll Rises to 1,400 Amid Af...",https://www.wsj.com/world/asia/afghan-quake-de...
1,2025-09-02-14-46-48 +0000,wapo,Taliban call for international aid as Afghan e...,https://www.washingtonpost.com/world/2025/09/0...
2,2025-09-02-14-46-32 +0000,bbc,Burkina Faso's parliament votes to outlaw homo...,https://www.bbc.com/news/articles/c2060er4062o...
3,2025-09-02-14-41-37 +0000,nyt,Trump’s Use of National Guard and Marines in C...,https://www.nytimes.com/2025/09/02/us/judge-ru...
4,2025-09-02-14-39-27 +0000,nyt,"How Protests, Tear Gas and Looting Roiled Indo...",https://www.nytimes.com/2025/09/02/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,45
159,china,17
53,new,12
10,earthquake,11
129,afghanistan,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
224,2025-09-01-20-19-52 +0000,nypost,India’s leader Modi cozies up to Putin and Xi ...,https://nypost.com/2025/09/01/world-news/india...,106
218,2025-09-01-20-43-57 +0000,nypost,Trump awarding Presidential Medal of Freedom t...,https://nypost.com/2025/09/01/us-news/trump-aw...,99
184,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...,93
188,2025-09-02-00-00-00 +0000,wsj,President Trump faces a challenging September ...,https://www.wsj.com/politics/policy/trumps-sep...,85
18,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,84


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
224,106,2025-09-01-20-19-52 +0000,nypost,India’s leader Modi cozies up to Putin and Xi ...,https://nypost.com/2025/09/01/world-news/india...
218,54,2025-09-01-20-43-57 +0000,nypost,Trump awarding Presidential Medal of Freedom t...,https://nypost.com/2025/09/01/us-news/trump-aw...
184,48,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...
266,43,2025-09-01-16-48-15 +0000,nypost,US companies doing business in China say they’...,https://nypost.com/2025/09/01/business/us-comp...
148,39,2025-09-02-05-11-55 +0000,nypost,"6-year-old boy in critical condition, 2 adults...",https://nypost.com/2025/09/02/us-news/2-adults...
26,36,2025-09-02-14-02-15 +0000,nypost,"Over 1,400 killed in Afghanistan earthquake as...",https://nypost.com/2025/09/02/world-news/over-...
27,36,2025-09-02-14-00-58 +0000,nypost,Trump’s use of National Guard during LA immigr...,https://nypost.com/2025/09/02/us-news/trumps-u...
122,36,2025-09-02-08-23-36 +0000,nypost,"Sudan landslide kills over 1,000 people and wi...",https://nypost.com/2025/09/02/world-news/sudan...
223,35,2025-09-01-20-20-50 +0000,nypost,"TikTok influencer and her family found dead, w...",https://nypost.com/2025/09/01/world-news/influ...
183,32,2025-09-02-00-13-00 +0000,nypost,"Rep. Jerrold Nadler, 78, will not seek re-elec...",https://nypost.com/2025/09/01/us-news/rep-jerr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
